In [1]:
import pandas as pd
from datetime import datetime

In [2]:
cmsa_df_filled = pd.read_csv('../data/cmsa_small_filled.csv')

In [3]:
cmsa_df_zero = pd.read_csv('../data/cmsa_small_fill_zero.csv')

In [4]:
vacation_df = pd.read_csv('../data/vacation.csv')

In [5]:
holiday_df = pd.read_csv('../data/holiday.csv')

In [6]:
covid_string = pd.read_csv('../data/covid-stringency.csv')

In [7]:
new_row = {'Day':'2022-01-01', 'stringency_index':63.89}
new_row2 = {'Day':'2022-01-02', 'stringency_index':63.89}
covid_string = covid_string.append(new_row, ignore_index=True)
covid_string = covid_string.append(new_row2, ignore_index=True)

In [8]:
def date_time_col(df):
    df["datetime_utc"] = pd.to_datetime(df["datetime"], utc = True)
    df["datetime"] = df["datetime_utc"].dt.tz_convert("Europe/Amsterdam")
    del df["datetime_utc"]
    df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df.sort_values(by = "datetime", ascending = True)
    return df

In [9]:
vacation_df = date_time_col(vacation_df)
holiday_df = date_time_col(holiday_df)

In [10]:
covid_string["Day"] = pd.to_datetime(covid_string["Day"], utc = True)
covid_string['Day'] = covid_string['Day'].dt.strftime('%Y-%m-%d %H:%M:%S')
covid_string = covid_string.rename(columns={"Day": "datetime"})

In [11]:
covid_string

,datetime,stringency_index
0,2020-09-01 00:00:00,50.93
1,2020-09-02 00:00:00,50.93
2,2020-09-03 00:00:00,50.93
3,2020-09-04 00:00:00,50.93
4,2020-09-05 00:00:00,50.93
...,...,...
484,2021-12-29 00:00:00,63.89
485,2021-12-30 00:00:00,63.89
486,2021-12-31 00:00:00,63.89
487,2022-01-01 00:00:00,63.89


In [12]:
cmsa_df_filled = cmsa_df_filled.set_index('datetime')
cmsa_df_zero = cmsa_df_zero.set_index('datetime')
vacation_df = vacation_df.set_index('datetime')
holiday_df = holiday_df.set_index('datetime')
covid_string = covid_string.set_index('datetime')

In [13]:
cmsa_df_filled.index = pd.DatetimeIndex(cmsa_df_filled.index)
cmsa_df_zero.index = pd.DatetimeIndex(cmsa_df_zero.index)

In [14]:
vacation_df.index = pd.DatetimeIndex(vacation_df.index)
holiday_df.index = pd.DatetimeIndex(holiday_df.index)
covid_string.index = pd.DatetimeIndex(covid_string.index)

In [15]:
start_date = '2020-09-01 00:00:00'
end_date = '2022-01-01 23:45:00'

In [16]:
cmsa_df_filled.shape

(46848, 3)

In [17]:
holiday_df = holiday_df.resample('15min').ffill(limit=96)
vacation_df = vacation_df.resample('15min').ffill(limit=96)
covid_string = covid_string.resample('15min').ffill(limit=96)

In [18]:
vacation_df = vacation_df[(vacation_df.index >= start_date) & (vacation_df.index <= end_date)]
holiday_df = holiday_df[(holiday_df.index >= start_date) & (holiday_df.index <= end_date)]
covid_string = covid_string[(covid_string.index >= start_date) & (covid_string.index <= end_date)]

In [19]:
cmsa_df_filled_merged = pd.concat([cmsa_df_filled, vacation_df, holiday_df, covid_string], axis=1)

In [20]:
cmsa_df_fill_zero_merged = pd.concat([cmsa_df_zero, vacation_df, holiday_df, covid_string], axis=1)

In [21]:
cmsa_df_filled_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index
datetime,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93
...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89


In [22]:
cmsa_df_fill_zero_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index
datetime,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93
...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89


In [23]:
hotels_airport = pd.read_csv('../data/hotels_airport.csv', sep=";")

In [24]:
hotels_airport["datetime"] = pd.to_datetime(hotels_airport["datetime"], utc = True)
hotels_airport['datetime'] = hotels_airport['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [25]:
hotels_airport = hotels_airport.set_index('datetime')

In [26]:
hotels_airport.index = pd.DatetimeIndex(hotels_airport.index)

In [27]:
hotels_airport = hotels_airport.resample('1D').ffill() 

In [28]:
hotels_airport = hotels_airport.resample('15min').ffill() 

In [29]:
hotels_airport = hotels_airport.reset_index()
new_row = {'datetime':'2022-01-01 23:45:00', 'hotel_gasten':342, 'hotel_overnachtingen':667,
           'airport_tot_passengers':3252433, 'airport_arrived_passengers':1657309,
           'airport_departed_passengers':1595124}
hotels_airport = hotels_airport.append(new_row, ignore_index=True)

In [30]:
hotels_airport = hotels_airport.set_index('datetime')
hotels_airport.index = pd.DatetimeIndex(hotels_airport.index)

In [31]:
hotels_airport = hotels_airport.resample('15min').ffill() 

In [32]:
hotels_airport

,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers
datetime,,,,,
2020-01-01 00:00:00,614,1216,5074773,2561003,2513770
2020-01-01 00:15:00,614,1216,5074773,2561003,2513770
2020-01-01 00:30:00,614,1216,5074773,2561003,2513770
2020-01-01 00:45:00,614,1216,5074773,2561003,2513770
2020-01-01 01:00:00,614,1216,5074773,2561003,2513770
...,...,...,...,...,...
2022-01-01 22:45:00,342,667,3252433,1657309,1595124
2022-01-01 23:00:00,342,667,3252433,1657309,1595124
2022-01-01 23:15:00,342,667,3252433,1657309,1595124


In [33]:
hotels_airport = hotels_airport[(hotels_airport.index >= start_date) & 
                                (hotels_airport.index <= end_date)]

In [34]:
hotels_airport

,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers
datetime,,,,,
2020-09-01 00:00:00,215,413,1337741,684061,653680
2020-09-01 00:15:00,215,413,1337741,684061,653680
2020-09-01 00:30:00,215,413,1337741,684061,653680
2020-09-01 00:45:00,215,413,1337741,684061,653680
2020-09-01 01:00:00,215,413,1337741,684061,653680
...,...,...,...,...,...
2022-01-01 22:45:00,342,667,3252433,1657309,1595124
2022-01-01 23:00:00,342,667,3252433,1657309,1595124
2022-01-01 23:15:00,342,667,3252433,1657309,1595124


In [35]:
cmsa_df_filled_merged = pd.concat([cmsa_df_filled_merged, hotels_airport], axis=1)

In [36]:
cmsa_df_fill_zero_merged = pd.concat([cmsa_df_fill_zero_merged, hotels_airport], axis=1)

In [37]:
cmsa_df_filled_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers
datetime,,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,215,413,1337741,684061,653680
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124


In [38]:
cmsa_df_fill_zero_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers
datetime,,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,215,413,1337741,684061,653680
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,215,413,1337741,684061,653680
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124


In [39]:
covid = pd.read_csv('../data/covid_cum.csv', sep=";")

In [40]:
# extract actual value
covid['covid_cases'] = covid['covid_cases'].diff()
covid['covid_hospital'] = covid['covid_hospital'].diff()
covid['covid_deaths'] = covid['covid_deaths'].diff()

In [41]:
covid["datetime"] = pd.to_datetime(covid["datetime"], utc = True)
covid['datetime'] = covid['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [42]:
covid = covid.set_index('datetime')
covid.index = pd.DatetimeIndex(covid.index)
covid = covid.resample('1D').ffill() 
covid = covid.resample('15min').ffill() 

In [43]:
covid = covid[(covid.index >= start_date) & (covid.index <= end_date)]

In [44]:
covid

,covid_cases,covid_hospital,covid_deaths
datetime,,,
2020-09-01 00:00:00,71.0,0.0,0.0
2020-09-01 00:15:00,71.0,0.0,0.0
2020-09-01 00:30:00,71.0,0.0,0.0
2020-09-01 00:45:00,71.0,0.0,0.0
2020-09-01 01:00:00,71.0,0.0,0.0
...,...,...,...
2022-01-01 22:45:00,1413.0,4.0,1.0
2022-01-01 23:00:00,1413.0,4.0,1.0
2022-01-01 23:15:00,1413.0,4.0,1.0


In [45]:
cmsa_df_filled_merged = pd.concat([cmsa_df_filled_merged, covid], axis=1)
cmsa_df_fill_zero_merged = pd.concat([cmsa_df_fill_zero_merged, covid], axis=1)

In [46]:
cmsa_df_filled_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers,covid_cases,covid_hospital,covid_deaths
datetime,,,,,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0


In [47]:
cmsa_df_fill_zero_merged

,GAWW-11,GAWW-12,GAWW-14,vacation_dummy,holiday_dummy,stringency_index,hotel_gasten,hotel_overnachtingen,airport_tot_passengers,airport_arrived_passengers,airport_departed_passengers,covid_cases,covid_hospital,covid_deaths
datetime,,,,,,,,,,,,,,
2020-09-01 00:00:00,104.0,113.0,32.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:15:00,73.0,109.0,30.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:30:00,84.0,88.0,27.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 00:45:00,95.0,99.0,26.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
2020-09-01 01:00:00,63.0,84.0,43.0,0,0,50.93,215,413,1337741,684061,653680,71.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 22:45:00,48.0,22.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0
2022-01-01 23:00:00,28.0,16.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0
2022-01-01 23:15:00,27.0,26.0,21.0,1,1,63.89,342,667,3252433,1657309,1595124,1413.0,4.0,1.0


In [48]:
# cmsa_df_filled_merged.to_csv('../data/cmsa_df_filled_merged.csv')

In [49]:
# cmsa_df_fill_zero_merged.to_csv('../data/cmsa_df_fill_zero_merged.csv')